In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

dataF = yf.download("EURUSD=X", start="2022-11-19", end="2023-01-16", interval='5m')
#dataF.iloc[:,:]

[*********************100%***********************]  1 of 1 completed


In [5]:
grid_distance = 0.005
midprice = 1.065

In [6]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=0.1)
grid


array([0.965, 0.97 , 0.975, 0.98 , 0.985, 0.99 , 0.995, 1.   , 1.005,
       1.01 , 1.015, 1.02 , 1.025, 1.03 , 1.035, 1.04 , 1.045, 1.05 ,
       1.055, 1.06 , 1.065, 1.07 , 1.075, 1.08 , 1.085, 1.09 , 1.095,
       1.1  , 1.105, 1.11 , 1.115, 1.12 , 1.125, 1.13 , 1.135, 1.14 ,
       1.145, 1.15 , 1.155, 1.16 , 1.165])

In [7]:
signal = [0]*len(dataF)
i=0
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High)<p and max(row.Low, row.High)>p:
            signal[i]=1
    i+=1

In [ ]:
dataF["signal"]=signal
dataF[dataF["signal"]==1]

,Open,High,Low,Close,Adj Close,Volume,signal
Datetime,,,,,,,
2022-11-17 00:00:00,1.040042,1.040042,1.039609,1.039609,1.039609,0,1
2022-11-17 08:05:00,1.040042,1.040042,1.038961,1.038961,1.038961,0,1
2022-11-17 08:15:00,1.039501,1.040583,1.039501,1.040366,1.040366,0,1
2022-11-17 08:20:00,1.040475,1.040475,1.039501,1.039501,1.039501,0,1
2022-11-17 08:35:00,1.040042,1.040475,1.039933,1.040150,1.040150,0,1
...,...,...,...,...,...,...,...
2023-01-13 10:25:00,1.085305,1.085776,1.084952,1.084952,1.084952,0,1
2023-01-13 10:30:00,1.085069,1.085069,1.084481,1.084716,1.084716,0,1
2023-01-13 13:45:00,1.080847,1.080964,1.079797,1.079797,1.079797,0,1


In [7]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high = dfpl.High, low = dfpl.Low, close = dfpl.Close, length = 16)
dfpl.dropna(inplace=True)

In [25]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class MyStrat(Strategy):
    mysize = 50
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.5*grid_distance #5*self.data.ATR[-1]
        TPSLRatio = 0.5

        if self.signal1==1 and len(self.trades)<=10000:             
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=50, margin=1/100, hedging=True, exclusive_orders=False)
stat = bt.run()
stat

Start                     2022-11-17 01:20:00
End                       2023-01-13 22:25:00
Duration                     57 days 21:05:00
Exposure Time [%]                   99.143792
Equity Final [$]                   136.018212
Equity Peak [$]                    156.586679
Return [%]                         172.036425
Buy & Hold Return [%]                4.233882
Return (Ann.) [%]                37364.619716
Volatility (Ann.) [%]            46309.602103
Sharpe Ratio                         0.806844
Sortino Ratio                      1089.28018
Calmar Ratio                      2194.580582
Max. Drawdown [%]                  -17.025859
Avg. Drawdown [%]                   -1.157491
Max. Drawdown Duration        9 days 23:10:00
Avg. Drawdown Duration        0 days 06:54:00
# Trades                                 1698
Win Rate [%]                        73.027091
Best Trade [%]                       0.867446
Worst Trade [%]                     -0.850432
Avg. Trade [%]                    

In [1]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False, 
plot_pl=False, plot_volume=False, plot_drawdown=False, smooth_equity=False, relative_equity=True, 
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)

NameError: name 'backtesting' is not defined

In [146]:
stat._trades.sort_values(by="EntryBar").head(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
24,-20,78,205,1.040799,1.035475,0.106497,0.005116,2022-11-15 07:50:00,2022-11-15 18:25:00,0 days 10:35:00
5,20,78,146,1.040799,1.045475,0.093503,0.004492,2022-11-15 07:50:00,2022-11-15 13:30:00,0 days 05:40:00
4,20,82,146,1.041016,1.045691,0.093501,0.004491,2022-11-15 08:10:00,2022-11-15 13:30:00,0 days 05:20:00
23,-20,82,205,1.041016,1.035691,0.106499,0.005115,2022-11-15 08:10:00,2022-11-15 18:25:00,0 days 10:15:00
3,20,96,146,1.039933,1.044825,0.097838,0.004704,2022-11-15 09:20:00,2022-11-15 13:30:00,0 days 04:10:00
22,-20,96,205,1.039933,1.034825,0.102162,0.004912,2022-11-15 09:20:00,2022-11-15 18:25:00,0 days 09:05:00
2,20,102,146,1.040150,1.045042,0.097835,0.004703,2022-11-15 09:50:00,2022-11-15 13:30:00,0 days 03:40:00
21,-20,102,205,1.040150,1.035042,0.102165,0.004911,2022-11-15 09:50:00,2022-11-15 18:25:00,0 days 08:35:00
1,20,106,146,1.040042,1.044825,0.095675,0.004600,2022-11-15 10:10:00,2022-11-15 13:30:00,0 days 03:20:00
20,-20,106,205,1.040042,1.034825,0.104325,0.005015,2022-11-15 10:10:00,2022-11-15 18:25:00,0 days 08:15:00
